Interpolant els resultats d'un llit elàstic, per poder posar a l'èbola la funcio i les dues primeres derivades

Aquest exemple usa els resultats de l'òptim de la Fase 1

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import optimize
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [2]:
llit = np.load("Llit_Exemple/Tensor.npy")

In [3]:
llit = llit.reshape((218*154, 3))

## 1.1 Donada la funció avaluada en una mesh, vull la funció pel continu

In [4]:
from scipy.interpolate import interp2d

In [5]:
%%time
f = interp2d(llit[:,0], llit[:,1], llit[:,2], kind='cubic')

CPU times: user 35.9 s, sys: 72 ms, total: 35.9 s
Wall time: 35.9 s


/usr/local/lib/python3.5/dist-packages/scipy/interpolate/_fitpack_impl.py:974: RuntimeWarning: The maximal number of iterations (20) allowed for finding smoothing
spline with fp=s has been reached. Probable cause: s too small.
(abs(fp-s)/s>0.001)
	kx,ky=3,3 nx,ny=47,43 m=33572 fp=0.000002 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


In [6]:
from tqdm import tqdm_notebook as t
llit = np.load("Llit_Exemple/Tensor.npy")
M = np.zeros(llit[:,:,0].shape)
for i in t(range(len(M))):
    for j in range(len(M[0])):
        M[i][j] = f(llit[i,j,0], llit[i,j,1])
np.max(M-llit[:,:,2]) #Sanity check, should be small

HBox(children=(IntProgress(value=0, max=218), HTML(value='')))

0.00018575120083597074

In [7]:
N = 500 #Granularitat de la mesh en la que evaluem gradient i hessiana abans d'interpolar

## 1.2 Donada la funció, vull poder evaluar el seu gradient en un punt arbitrari

In [8]:
f

In [9]:
f_vec = lambda x: f(*x)

In [10]:
X = np.linspace(-2.5, 2.5, N)
Y = np.linspace(-2.5, 2.5, N)
M = np.zeros((N,N,2))
for i in t(range(N)):
    for j in range(N):
        M[i][j] = optimize.approx_fprime([X[i],X[j]],f_vec,epsilon = 1e-10)

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

In [11]:
#friendship with interp2d ended, RectBivariateSpline is my new friend
from scipy.interpolate import RectBivariateSpline
dfx = RectBivariateSpline(X,Y,M[:,:,0])
dfy = RectBivariateSpline(X,Y,M[:,:,1])

In [12]:
def gradf(x,y):
    return np.array([dfx(x,y)[0], dfy(x,y)[0]]).reshape(2)

In [13]:
gradf(2,3)

array([ -7.17734996e-10,   0.00000000e+00])

## 1.2 Donada la funció, vull poder evaluar la seva Hessiana en un punt arbitrari

In [14]:
dfx_vec = lambda x: dfx(*x)
dfy_vec = lambda x: dfy(*x)

In [15]:
X = np.linspace(-2.5, 2.5, N)
Y = np.linspace(-2.5, 2.5, N)
M = np.zeros((N,N,2))
for i in t(range(N)):
    for j in range(N):
        M[i][j] = optimize.approx_fprime([X[i],X[j]],dfx_vec,epsilon = 1e-10)
        
dfxx = RectBivariateSpline(X,Y,M[:,:,0])
dfxy = RectBivariateSpline(X,Y,M[:,:,1])

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

In [16]:
X = np.linspace(-2.5, 2.5, N)
Y = np.linspace(-2.5, 2.5, N)
M = np.zeros((N,N,2))
for i in t(range(N)):
    for j in range(N):
        M[i][j] = optimize.approx_fprime([X[i],X[j]],dfy_vec,epsilon = 1e-10)
        
dfyx = RectBivariateSpline(X,Y,M[:,:,0])
dfyy = RectBivariateSpline(X,Y,M[:,:,1])

HBox(children=(IntProgress(value=0, max=500), HTML(value='')))

In [17]:
def Hessf(x,y):
    return np.array([[dfxx(x,y)[0], dfxy(x,y)[0]], [dfyx(x,y)[0], dfyy(x,y)[0]]]).reshape(2,2)

In [21]:
Hessf

<function __main__.Hessf>

In [20]:
Hessf(0.2,0.3)

array([[ 0.27330879, -0.93730426],
       [-0.93724756, -0.08185295]])

## Ara ens hem de guardar els resultats per poder-li passar les funcions a l'ebola

In [24]:
import pickle

pickle.dump(f, open("f.p", 'wb'))
pickle.dump(dfx, open("dfx.p", 'wb'))
pickle.dump(dfy, open("dfy.p", 'wb'))
pickle.dump(dfxx, open("dfxx.p", 'wb'))
pickle.dump(dfxy, open("dfxy.p", 'wb'))
pickle.dump(dfyx, open("dfyx.p", 'wb'))
pickle.dump(dfyy, open("dfyy.p", 'wb'))
